This code resizes "test" and "train" images based on their longer sides found under PATH into DEST with the same directory hierarchy.
You can give multiple sizes to create in one run as demonstrated in the commented line.
The code is based on fastai's [script](https://github.com/fastai/imagenet-fast/blob/master/scripts/resize.py).

In [ ]:
from PIL import Image
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
cpus = multiprocessing.cpu_count()
cpus = min(48,cpus-1)

ds_path = Path('/media/nofreewill/Datasets_nvme/kaggle/bms-molecular-translation')
PATH = ds_path
DEST = ds_path/'resized'
#szs = (224, 512, 1024, )
szs = (224, )

def resize_img(im, fn, sz):
    new_fn = DEST/str(sz)/fn.relative_to(PATH)
    new_fn.parent.mkdir(parents=True, exist_ok=True)
    w,h = im.size
    ratio = max(h/sz,w/sz)
    im = im.resize((int(w/ratio), int(h/ratio)), resample=Image.BICUBIC)
    im.save(new_fn)

def resizes(fn):
    im = Image.open(fn)
    for sz in szs: resize_img(im, fn, sz)

def resize_imgs(p):
    files = p.rglob('**/*.png')
    with ProcessPoolExecutor(cpus) as e: e.map(resizes, files)


for ds in ('test','train'): resize_imgs(PATH/ds)